## First we create the same dataframe as before

In [1]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = req.get(link).text
wiki = bs(data, "html")

In [3]:
table = wiki.find('table')

In [4]:
cn = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = cn)

In [5]:
for tr in table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [6]:
df=df[df['Borough']!='Not assigned']
i = 0
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1

In [7]:
df = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [8]:
df = df.dropna()
df = df[(df.Postalcode != 'Not assigned' )]

In [9]:
#Shape of the df 
print(df.shape)

(103, 3)


## Now we add the geographical coordinates of each postal code

In [10]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2020-11-19 07:26:38--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.63.96.176, 169.63.96.194
Connecting to cocl.us (cocl.us)|169.63.96.176|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2020-11-19 07:26:39--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.63.96.176|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-11-19 07:26:40--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-11-19 07:26:40--  https://ibm.box.com/public/static/9afzr

In [11]:
dfgeo = pd.read_csv('GeoCord.csv')
dfgeo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [12]:
df = pd.merge(df,dfgeo[["Postal Code","Latitude","Longitude"]],left_on="Postalcode", right_on="Postal Code", how = "left")
df.drop(columns = "Postal Code")

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
